In [22]:
import numpy as np
import yaml
import torch
from data.phylogeny import PhylogenyCUB
from data.analysis_utils import PhylogenyMapper, get_phylomapper_from_config

In [17]:
phylo = PhylogenyCUB('/home/harishbabu/data/phlyogenyCUB')
phyloDistances_string = '0.93,0.83,0.63'

In [ ]:

phyloDistances = [float(x) for x in phyloDistances_string.split(',')[::-1]] + [1]
num_levels = len(phyloDistances)

ancestor_lvl_to_spc_groups = {} # maps ancestor levels (int) to spc groups (lists of species names)
yaml_dict = {}
for ancestor_lvl, phylo_dist in enumerate(phyloDistances[:-1]):
    ancestor_lvl_to_spc_groups[ancestor_lvl] = [{spc_group[0]: spc_group} for spc_group in phylo.get_species_groups(1-phylo_dist)]
    if len(yaml_dict) == 0:
        for spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl]:
            representative = spc_group[0]
            yaml_dict[representative] = {}
    else:
        for representative_spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl]:

            pass
        ancestor_lvl_to_spc_groups[ancestor_lvl-1]

lb_level_dict = [{spc_group[0]: spc_group} for spc_group in ancestor_lvl_to_spc_groups[num_levels-2]]

for ancestor_lvl in range(num_levels-2, -1, -1):
    pass

In [31]:
for i in range(3):
    phylomapper = get_phylomapper_from_config(PhylogenyCUB('/home/harishbabu/data/phlyogenyCUB'), phyloDistances_string, level=i)
    mapped_truth = phylomapper.get_mapped_truth(torch.tensor([189]))
    print(mapped_truth)

tensor([3])
tensor([3])
tensor([73])


In [4]:
import os
import shutil

import torch
import torch.utils.data
import transforms
import torchvision.datasets as datasets
import argparse
from helpers import makedir, adjust_learning_rate
import model
import push
import train_and_test as tnt
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function, img_size
from node import Node
import time
import numpy as np
from data.phylogeny import PhylogenyCUB

# root = Node("root")
# root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
# root.add_children_to('animal',['non_primate','primate'])
# root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
# root.add_children_to('primate',['capuchin','gibbon','orangutan'])
# root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
# root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
# root.add_children_to('weapon',['assault_rifle','rifle'])

In [19]:
def construct_phylo_tree(phylogeny_path, phyloDistances_string):
    phylo = PhylogenyCUB(phylogeny_path) # '/home/harishbabu/data/phlyogenyCUB'
    root = Node("root")
    phyloDistances = [float(x) for x in phyloDistances_string.split(',')[::-1]] + [1]
    num_levels = len(phyloDistances)

    ances_lvl_tag_prefix = '_lvl'

    ancestor_lvl_to_spc_groups = {} # maps ancestor levels (int) to spc groups (dict mapping representative_species to a list of species)
    for ancestor_lvl, phylo_dist in enumerate(phyloDistances):
        if ancestor_lvl == len(phyloDistances)-1:
            ancestor_lvl_to_spc_groups[ancestor_lvl] = {spc_group[0]: spc_group \
                                                        for spc_group in phylo.get_species_groups(1-phylo_dist)}
        else:
            ancestor_lvl_to_spc_groups[ancestor_lvl] = {(spc_group[0] + ances_lvl_tag_prefix + str(ancestor_lvl)): spc_group \
                                                        for spc_group in phylo.get_species_groups(1-phylo_dist)}
        if ancestor_lvl == 0:
            children_list = []
            for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl].items():
                children_list.append(representative)
            root.add_children(children_list)
        else:
            prev_level_representatives = [representative for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl - 1].items()]
            prev_level_representative_to_children = {representative: [] for representative in prev_level_representatives}
            for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl].items():
                for prev_lvl_rep in prev_level_representatives:
                    if representative.split(ances_lvl_tag_prefix)[0] in ancestor_lvl_to_spc_groups[ancestor_lvl - 1][prev_lvl_rep]:
                        prev_level_representative_to_children[prev_lvl_rep].append(representative)
                        break
            
            for prev_lvl_rep, children in prev_level_representative_to_children.items():
                root.add_children_to(prev_lvl_rep, children)

    return root


In [20]:
# root = construct_phylo_tree(phylogeny_path='/home/harishbabu/data/phlyogenyCUB', 
#                      phyloDistances_string='0.93,0.83,0.63')
root = construct_phylo_tree(phylogeny_path='/home/harishbabu/data/phlyogenyCUB/1_tree-consensus-Hacket-AllSpecies-modified_cub-names_v1.phy',
                          phyloDistances_string='0.93,0.83,0.63')
print(getattr(root, 'children_names')())

['cub_001_Black_footed_Albatross_lvl0', 'cub_004_Groove_billed_Ani_lvl0', 'cub_005_Crested_Auklet_lvl0', 'cub_009_Brewer_Blackbird_lvl0', 'cub_018_Spotted_Catbird_lvl0', 'cub_019_Gray_Catbird_lvl0', 'cub_023_Brandt_Cormorant_lvl0', 'cub_029_American_Crow_lvl0', 'cub_036_Northern_Flicker_lvl0', 'cub_037_Acadian_Flycatcher_lvl0', 'cub_046_Gadwall_lvl0', 'cub_050_Eared_Grebe_lvl0', 'cub_067_Anna_Hummingbird_lvl0', 'cub_085_Horned_Lark_lvl0', 'cub_086_Pacific_Loon_lvl0', 'cub_185_Bohemian_Waxwing_lvl0']


In [25]:
root.children_names()
root.get_node('cub_004_Groove_billed_Ani' + '_lvl2').children_names()

['cub_004_Groove_billed_Ani']

Testing unwrap_names_of_joint

In [11]:
def unwrap_names_of_joint(names):
    # poorly written function for unwrapping nested lists up to depth 3 -- will cause bug if class hierarchy is too deep
    def contains_list(names):
        for item in names:
            if type(item) is list:
                return True
        return False

    if not contains_list(names):
        return names

    new_list = []
    for item in names:
        if type(item) is not list:
            new_list += [item]
        else:
            new_list += unwrap_names_of_joint(item)
    return new_list

test_list = [[[1, 2], [3], [[4, 5, [6, [8, 9]], [10]]]]]
unwrap_names_of_joint(test_list)

[1, 2, 3, 4, 5, 6, 8, 9, 10]

In [17]:
def unwrap_names_of_joint(names):
    # poorly written function for unwrapping nested lists up to depth 3 -- will cause bug if class hierarchy is too deep
    new_list = []
    for item in names:
        if type(item) is not list:
            new_list.append(item)
        else:
            for subitem in item:
                if type(subitem) is not list:
                    new_list.append(subitem)
                else:
                    for subsubitem in subitem:
                        if type(subsubitem) is not list:
                            new_list.append(subsubitem)
                        else:
                            for subsubsubitem in subsubitem:
                                if type(subsubsubitem) is not list:
                                    new_list.append(subsubsubitem)

    return new_list
test_list = [[1, 2], [3], [[4, 5, [6, [8, 9]], [10]]]]
test_list = [[1], 2, [[3, 4]], [[[[5, 6]]]]]
unwrap_names_of_joint(test_list)

[1, 2, 3, 4]

In [27]:
dir(root)
# setattr()
getattr(root, 'children_names')()

['001.Black_footed_Albatross_lvl0',
 '004.Groove_billed_Ani_lvl0',
 '005.Crested_Auklet_lvl0',
 '009.Brewer_Blackbird_lvl0',
 '018.Spotted_Catbird_lvl0',
 '019.Gray_Catbird_lvl0',
 '023.Brandt_Cormorant_lvl0',
 '029.American_Crow_lvl0',
 '036.Northern_Flicker_lvl0',
 '037.Acadian_Flycatcher_lvl0',
 '046.Gadwall_lvl0',
 '050.Eared_Grebe_lvl0',
 '067.Anna_Hummingbird_lvl0',
 '085.Horned_Lark_lvl0',
 '086.Pacific_Loon_lvl0',
 '185.Bohemian_Waxwing_lvl0']

In [17]:
from node import Node
# import time
# import numpy as np
from data.phylogeny import PhylogenyCUB

def construct_phylo_tree(phylogeny_path, phyloDistances_string):
    phylo = PhylogenyCUB(phylogeny_path) # '/home/harishbabu/data/phlyogenyCUB'
    root = Node("root")
    phyloDistances = [float(x) for x in phyloDistances_string.split(',')[::-1]] + [1]
    num_levels = len(phyloDistances)

    ances_lvl_tag_prefix = '_lvl'

    ancestor_lvl_to_spc_groups = {} # maps ancestor levels (int) to spc groups (dict mapping representative_species to a list of species)
    for ancestor_lvl, phylo_dist in enumerate(phyloDistances[:-1]):
        ancestor_lvl_to_spc_groups[ancestor_lvl] = {(spc_group[0] + ances_lvl_tag_prefix + str(ancestor_lvl)): spc_group \
                                                    for spc_group in phylo.get_species_groups(1-phylo_dist)}
        
        if ancestor_lvl == 0:
            children_list = []
            for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl].items():
                children_list.append(representative)
            root.add_children(children_list)
        else:
            prev_level_representatives = [representative for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl - 1].items()]
            prev_level_representative_to_children = {representative: [] for representative in prev_level_representatives}
            for representative, spc_group in ancestor_lvl_to_spc_groups[ancestor_lvl].items():
                for prev_lvl_rep in prev_level_representatives:
                    if representative.split(ances_lvl_tag_prefix)[0] in ancestor_lvl_to_spc_groups[ancestor_lvl - 1][prev_lvl_rep]:
                        prev_level_representative_to_children[prev_lvl_rep].append(representative)
                        break
            
            for prev_lvl_rep, children in prev_level_representative_to_children.items():
                root.add_children_to(prev_lvl_rep, children)

    return root

In [18]:
root = construct_phylo_tree(phylogeny_path='/home/harishbabu/data/phlyogenyCUB/1_tree-consensus-Hacket-AllSpecies-modified_cub-names_v1.phy',
                          phyloDistances_string='0.93,0.83,0.63')
# root = construct_phylo_tree(phylogeny_path='/home/harishbabu/data/phlyogenyCUB/1_tree-consensus-Hacket-AllSpecies-cub-names.phy',
#                           phyloDistances_string='0.93,0.83,0.63')
# print(getattr(root, 'children_names')())
for child in getattr(root, 'children_names')():
    print(child)

cub_001_Black_footed_Albatross_lvl0
cub_004_Groove_billed_Ani_lvl0
cub_005_Crested_Auklet_lvl0
cub_009_Brewer_Blackbird_lvl0
cub_018_Spotted_Catbird_lvl0
cub_019_Gray_Catbird_lvl0
cub_023_Brandt_Cormorant_lvl0
cub_029_American_Crow_lvl0
cub_036_Northern_Flicker_lvl0
cub_037_Acadian_Flycatcher_lvl0
cub_046_Gadwall_lvl0
cub_050_Eared_Grebe_lvl0
cub_067_Anna_Hummingbird_lvl0
cub_085_Horned_Lark_lvl0
cub_086_Pacific_Loon_lvl0
cub_185_Bohemian_Waxwing_lvl0


In [5]:
# root.get_node('cub_046_Gadwall' + '_lvl0').children_names()

In [16]:
phylo = PhylogenyCUB('/home/harishbabu/data/phlyogenyCUB/1_tree-consensus-Hacket-AllSpecies-modified_cub-names_v1.phy')
# phylo = PhylogenyCUB('/home/harishbabu/data/phlyogenyCUB/1_tree-consensus-Hacket-AllSpecies-cub-names.phy')

# for node in phylo.tree.iter_leaves():
#     print(node.name)
    # node.name = "_".join(['cub'] + node.name.split('.'))

spc_grps = phylo.get_species_groups(1-0.63)
len(spc_grps)

16